In [4]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/falcon-7b-instruct-GPTQ"
# You could also download the model locally, and access it there
# model_name_or_path = "/path/to/TheBloke_falcon-7b-instruct-GPTQ"

model_basename = "gptq_model-4bit-64g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,d
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        use_triton=use_triton,
        quantize_config=None)

prompt = "Tell me about AI"
prompt_template=f'''A helpful assistant who helps the user with any questions asked.
User: {prompt}
Assistant:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt')
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline
# Note that if you use pipeline, you will see a spurious error message saying the model type is not supported
# This can be ignored!  Or you can hide it with the following logging line:
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Anubhav194010\AppData\Local\Temp\1\ipykernel_11148\2037443758.py:15 in <cell line: 15>  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Anubhav194010\\AppData\\Local\\Temp\\1\\ipykernel_11148\\2037443758.py'              │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\auto_gptq\modeling\auto.py:82 in              │
│ from_quantized                                                                                   │
│                                                                                                  │
│    79 │   │   model_type = check_and_get_model_type(save_dir or model_name_or_path, trust_remo   │
│    80 │   │   quant_func = GPTQ_CAUSAL_LM_MODEL_MAP[model_type].from_quantized                   │
│    81 │   │   keywords = {key: kwargs[key] for key in signature(quant_func).parameters if key    │
│ ❱  82 │   │   return quant_func(                                                                 │
│    83 │   │   │   model_name_or_path=model_name_or_path,                                         │
│    84 │   │   │   save_dir=save_dir,                                                             │
│    85 │   │   │   device_map=device_map,                                                         │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\auto_gptq\modeling\_base.py:773 in            │
│ from_quantized                                                                                   │
│                                                                                                  │
│   770 │   │   if low_cpu_mem_usage:                                                              │
│   771 │   │   │   make_sure_no_tensor_in_meta_device(model, use_triton, quantize_config.desc_a   │
│   772 │   │                                                                                      │
│ ❱ 773 │   │   accelerate.utils.modeling.load_checkpoint_in_model(                                │
│   774 │   │   │   model,                                                                         │
│   775 │   │   │   checkpoint=model_save_name,                                                    │
│   776 │   │   │   device_map=device_map,                                                         │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\accelerate\utils\modeling.py:1042 in          │
│ load_checkpoint_in_model                                                                         │
│                                                                                                  │
│   1039 │   check_tied_parameters_on_same_device(tied_params, device_map)                         │
│   1040 │                                                                                         │
│   1041 │   if offload_folder is None and device_map is not None and "disk" in device_map.values  │
│ ❱ 1042 │   │   raise ValueError(                                                                 │
│   1043 │   │   │   "At least one of the model submodule will be offloaded to disk, please pass   │
│   1044 │   │   )                                                                                 │
│   1045 │   elif offload_folder is not None and device_map is not None and "disk" in device_map.  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: At least one of the model submodule will be offlo

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/xgen-7b-8k-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Salesforce/xgen-7b-8k-base", torch_dtype=torch.bfloat16)
inputs = tokenizer("The world is", return_tensors="pt")
sample = model.generate(**inputs, max_length=128)
print(tokenizer.decode(sample[0]))


c:\Users\Anubhav194010\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anubhav194010\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


A new version of the following files was downloaded from https://huggingface.co/Salesforce/xgen-7b-8k-base:
- tokenization_xgen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Anubhav194010\AppData\Local\Temp\1\ipykernel_11148\2419967694.py:5 in <cell line: 5>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Anubhav194010\\AppData\\Local\\Temp\\1\\ipykernel_11148\\2419967694.py'              │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\transformers\models\auto\auto_factory.py:484  │
│ in from_pretrained                                                                               │
│                                                                                                  │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 484 │   │   │   return model_class.from_pretrained(                                            │
│   485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                                                              │
│   487 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\transformers\modeling_utils.py:2585 in        │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2582 │   │   # We'll need to download and cache each checkpoint shard if the checkpoint is sh  │
│   2583 │   │   if is_sharded:                                                                    │
│   2584 │   │   │   # rsolved_archive_file becomes a list of files that point to the different c  │
│ ❱ 2585 │   │   │   resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(         │
│   2586 │   │   │   │   pretrained_model_name_or_path,                                            │
│   2587 │   │   │   │   resolved_archive_file,                                                    │
│   2588 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\transformers\utils\hub.py:958 in              │
│ get_checkpoint_shard_files                                                                       │
│                                                                                                  │
│    955 │   for shard_filename in tqdm(shard_filenames, desc="Downloading shards", disable=not s  │
│    956 │   │   try:                                                                              │
│    957 │   │   │   # Load from URL                                                               │
│ ❱  958 │   │   │   cached_filename = cached_file(                                                │
│    959 │   │   │   │   pretrained_model_name_or_path,                                            │
│    960 │   │   │   │   shard_filename,                                                           │
│    961 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ c:\Users\Anubhav194010\Anaconda3\lib\site-packages\transfor

In [ ]:
def generate_response_falcon(txt):
    repo_id = "tiiuae/falcon-7b"

    falcon_llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token,
                     repo_id=repo_id, task='text2text-generation',
                     model_kwargs={"temperature":0.0001, "max_new_tokens":200, "do_sample":False, "num_return_sequences":1, "repetition_penalty":100})
    # print(txt)
    sum_template = """Summarize the following text as an essay in not more than 100 words.:{text}
    Answer: """
    # print("\n\n",sum_template)
    
    sum_prompt = PromptTemplate(template=sum_template, input_variables=["text"])
    
    sum_llm_chain = LLMChain(prompt=sum_prompt, llm=falcon_llm)    
    summary = sum_llm_chain.run(txt)
    print(summary)
    wrapped_text = textwrap.fill(summary, width=100, break_long_words=False, replace_whitespace=False)
    # print(wrapped_text)
    return str(wrapped_text)

In [1]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [2]:
chat = ChatAnthropic()

ValidationError: 1 validation error for ChatAnthropic
__root__
  Did not find anthropic_api_key, please add an environment variable `ANTHROPIC_API_KEY` which contains it, or pass  `anthropic_api_key` as a named parameter. (type=value_error)